data_schema = {
    "time": {
        "description": "Timestamp in the format 'YYYY-MM-DD HH:MM:SS.FFF'",
        "type": "string",
        "pattern": "^\\d{4}-\\d{2}-\\d{2} \\d{2}:\\d{2}:\\d{2}\\.\\d{3}$"
    },
    "mood": {
        "description": "The mood scored by the user on a scale of 1-10",
        "type": "int",
        "range": (1, 10)
    },
    "circumplex.arousal": {
        "description": "The arousal scored by the user, on a scale between -2 to 2",
        "type": "int",
        "range": (-2, 2)
    },
    "circumplex.valence": {
        "description": "The valence scored by the user, on a scale between -2 to 2",
        "type": "int",
        "range": (-2, 2)
    },
    "activity": {
        "description": "Activity score of the user (number between 0 and 1)",
        "type": "float",
        "range": (0, 1)
    },
    "screen": {
        "description": "Duration of screen activity (time)",
        "type": "float",
        "range": None
    },
    "call": {
        "description": "Call made (indicated by a 1)",
        "type": "int",
        "range": (0, 1)
    },
    "sms": {
        "description": "SMS sent (indicated by a 1)",
        "type": "int",
        "range": (0, 1)
    },
    "appCat.builtin": {
        "description": "Duration of usage of builtin apps (time)",
        "type": "float",
        "range": None
    },
    "appCat.communication": {
        "description": "Duration of usage of communication apps (time)",
        "type": "float",
        "range": None
    },
    "appCat.entertainment": {
        "description": "Duration of usage of entertainment apps (time)",
        "type": "float",
        "range": None
    },
    "appCat.finance": {
        "description": "Duration of usage of finance apps (time)",
        "type": "float",
        "range": None
    },
    "appCat.game": {
        "description": "Duration of usage of game apps (time)",
        "type": "float",
        "range": None
    },
    "appCat.office": {
        "description": "Duration of usage of office apps (time)",
        "type": "float",
        "range": None
    },
    "appCat.other": {
        "description": "Duration of usage of other apps (time)",
        "type": "float",
        "range": None
    },
    "appCat.social": {
        "description": "Duration of usage of social apps (time)",
        "type": "float",
        "range": None
    },
    "appCat.travel": {
        "description": "Duration of usage of travel apps (time)",
        "type": "float",
        "range": None
    },
    "appCat.unknown": {
        "description": "Duration of usage of unknown apps (time)",
        "type": "float",
        "range": None
    },
    "appCat.utilities": {
        "description": "Duration of usage of utilities apps (time)",
        "type": "float",
        "range": None
    },
    "appCat.weather": {
        "description": "Duration of usage of weather apps (time)",
        "type": "float",
        "range": None
    }
}


import json
# 创建一个新的json对象，其中只有"time"键
new_data_schema = {"time": data_schema["time"]}

# 创建一个新的"variable"对象，其中包含除"time"之外的所有键值对
new_data_schema["variable"] = {key: value for key, value in data_schema.items() if key != "time"}

# new_data_schema现在是修改后的json
# 将new_data_schema保存为JSON
with open("../Assignment1/threshold.json", 'w') as f:
    json.dump(new_data_schema, f)

In [1]:
import pandas as pd
# 数据
data=pd.read_csv('../Assignment1//dataset_mood_smartphone.csv')
data.set_index(data.columns[0],inplace=True)
data.reset_index(inplace=True)
data.index += 1
data.head(5)

,Unnamed: 0,id,time,variable,value
1,1,AS14.01,2014-02-26 13:00:00.000,mood,6.0
2,2,AS14.01,2014-02-26 15:00:00.000,mood,6.0
3,3,AS14.01,2014-02-26 18:00:00.000,mood,6.0
4,4,AS14.01,2014-02-26 21:00:00.000,mood,7.0
5,5,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [15]:
import numpy as np
print(data['value'].iloc[5708])
print(data['value'].iloc[5708] is np.NaN)

nan
False


In [27]:
import re

# 记录不符合规定的行索引

data_schema=pd.read_json('../Assignment1//threshold.json')
# 遍历每一行
def clean_raw(data, data_schema):
    invalid_rows = []
    error_types = []
    for index, row in data.iterrows():
    # 检查时间是否符合规定
        if not re.match(data_schema['time']['pattern'], row['time']):
            invalid_rows.append(index)
            error_types.append('wrong_time')
            continue

    # 检查变量是否符合规定
        variable = row['variable']
        value = row['value']
        if variable in data_schema['variable']:
            var_schema = data_schema['variable'][variable]
            if var_schema['range'] is not None:
                min_value, max_value = var_schema['range']
                if not min_value <= value <= max_value:
                    if pd.isnull(value):
                        invalid_rows.append(index)
                        error_types.append('missing_value')
                    else:
                        invalid_rows.append(index)
                        error_types.append('out_of_range')
        else:
            invalid_rows.append(index)
            error_types.append('missing_variable')

    print(len(invalid_rows))
    print("invalid row id:", invalid_rows)
    print("error types:", error_types)


In [35]:
import pandas as pd
from io import StringIO

data_str = """
"","id","time","variable","value"
"5639","AS14.33","2014-99-30 19:00:00.000","mood",8
"5640","AS14.33","2014-05-30 20:00:00.000","NA",6
"5641","AS14.33","2014-05-31 12:00:00.000","mood",110
"5642","AS14.01","2014-02-26 13:00:00.000","circumplex.arousal",NA
"5643","AS14.01","206 15:00:00.000","circumplex.arousal",-1
"5644","AS14.01","2014-02-26 18:00:00.000","circumplex.arousal",2000
"""

data = pd.read_csv(StringIO(data_str), quotechar='"', skipinitialspace=True)
print(data)

   Unnamed: 0       id                     time            variable   value
0        5639  AS14.33  2014-99-30 19:00:00.000                mood     8.0
1        5640  AS14.33  2014-05-30 20:00:00.000                 NaN     6.0
2        5641  AS14.33  2014-05-31 12:00:00.000                mood   110.0
3        5642  AS14.01  2014-02-26 13:00:00.000  circumplex.arousal     NaN
4        5643  AS14.01         206 15:00:00.000  circumplex.arousal    -1.0
5        5644  AS14.01  2014-02-26 18:00:00.000  circumplex.arousal  2000.0


In [36]:
clean_raw(data, data_schema)

5
invalid row id: [1, 2, 3, 4, 5]
error types: ['missing_variable', 'out_of_range', 'missing_value', 'wrong_time', 'out_of_range']
